In order to run this, you need to install the following libraries:

1) **Vader** - https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f
   
   *pip install vaderSentiment*
   
   This is used for the Sentiment Analysis
   
2) **Boto3** - https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.03.html#GettingStarted.Python.03.01

   *pip install boto3*
   
   This is used to read/write data into DynamoDB objects. Note you have to setup AWSCLI below first as it uses that for
   the connection/user details.
   
3) **AWSCLI** - https://sysadmins.co.za/interfacing-amazon-dynamodb-with-python-using-boto3/

   *pip install awscli*
   
   This is used to setup the connection/configuration parameters needed to access the DynamoDB objects. Sanjeev had provided 
   the details in his email and the instructions show you what to do under Lets get started

In [1]:
import pandas as pd
import datetime as dt
import praw
import boto3
import json
import decimal

from decimal import Decimal
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In order to use the PRAW api, follow the directions here https://www.storybench.org/how-to-scrape-reddit-with-python/
Note this involves creating a Reddit Account and a Reddit App ID which the instructions guide you through

In [2]:
PERSONAL_USE_SCRIPT_14_CHARS = 'your personal use'
SECRET_KEY_27_CHARS = 'your secret key'
YOUR_APP_NAME = 'your app name'
YOUR_REDDIT_USER_NAME = 'your reddit account'
YOUR_REDDIT_LOGIN_PASSWORD = 'your reddit password'

In [24]:
# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if abs(o) % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

# This class is used to handle all of the interactions (uploading/downloading tables etc) from
# the DynamoDB tables setup for the project
class DynamoDBEngine:
    def __init__(self):
        #This creates the dynamoDB object that points to the location of the database
        #Note this requires the AWSCLI connection details to be setup
        self.dynamodb = boto3.resource('dynamodb', region_name='us-east-1', endpoint_url="https://dynamodb.us-east-1.amazonaws.com")
    
    #This takes the comments panda dataframe and uploads comments to the UserComments
    #table in DynamoDB one row at a time
    def uploadComments(self, comment_df):        
        table = self.dynamodb.Table('UserComments')
        
        for row in comment_df.itertuples():
            response = table.put_item(
                Item={
                    'comment_id': row.comment_id,
                    'story_id': row.story_id,
                    'comment_author': row.comment_author,
                    'comment_body': row.comment_body,
                    'negative_sa_score': Decimal(str(row.negative_sa_score)),
                    'neutral_sa_score': Decimal(str(row.neutral_sa_score)),
                    'positive_sa_score': Decimal(str(row.positive_sa_score)),
                    'compound_sa_score': Decimal(str(row.compound_sa_score))
                })
            
            if response['ResponseMetadata']['HTTPStatusCode'] != 200:
                return False
        
        return True
    
    #This takes the comments panda dataframe and uploads comments to the UserComments
    #table in DynamoDB one row at a time
    def uploadStories(self, story_df):
        table = self.dynamodb.Table('UserStories')
        
        for row in story_df.itertuples():
            response = table.put_item(
                Item={
                    'story_id': row.story_id,
                    'title': row.title,
                    'author': row.author,
                    'body': row.body
                })
            
            if response['ResponseMetadata']['HTTPStatusCode'] != 200:
                return False
        
        return True       

# This class is used to handle all of the interactions (downloading stories/comments) from
# the NoSleep Subreddit
class NoSleepRecommender:
    def __init__(self):
        self.reddit = praw.Reddit(client_id=PERSONAL_USE_SCRIPT_14_CHARS,
                                  client_secret=SECRET_KEY_27_CHARS,
                                  password=YOUR_REDDIT_LOGIN_PASSWORD,
                                  user_agent=YOUR_APP_NAME,
                                  username=YOUR_REDDIT_USER_NAME)

        #print(self.reddit.user.me())

        self.subreddit = self.reddit.subreddit('nosleep')

        self.stories_dict = {"story_id": [],
                        "title": [],
                        "author": [],
                        "body": []}
        self.comments_dict = {"comment_id": [],
                         "story_id": [],
                         "comment_author": [],
                         "comment_body": [],
                         "negative_sa_score": [],
                         "neutral_sa_score": [],
                         "positive_sa_score": [],
                         "compound_sa_score": []}
    
    def loadComments(self):
        self.comment_df = pd.read_csv('user_comments.csv')

    def loadStories(self):
        self.story_df = pd.read_csv('user_stories.csv')
        self.story_df = self.story_df.fillna(' ')
        
    def loadAll(self):
        self.loadComments()
        self.loadStories()
        
    def returnComments(self):
        return self.comment_df
    
    def returnStories(self):
        return self.story_df    
    
    def saveComments(self):
        self.comment_df.to_csv (r'user_comments.csv', index = False, header=True)
        
    def saveStories(self):
        self.story_df.to_csv (r'user_stories.csv', index = False, header=True)
        
    def saveAll(self):
        self.saveComments()
        self.saveStories()
        
    def getStories(self):
        analyser = SentimentIntensityAnalyzer()

        my_subreddit = self.subreddit.hot(limit=10)
        for submission in my_subreddit:
            self.stories_dict["title"].append(submission.title)
            self.stories_dict["body"].append(submission.selftext)
            self.stories_dict["author"].append(submission.author)
            self.stories_dict["story_id"].append(submission.id)
            
            submission.comments.replace_more(limit=None)
            all_comments = submission.comments.list()            
            
            for comment in all_comments:
                #This does the sentiment analysis and returns the
                #scores obtained for the comment. A compound score is a
                #one-dimensional assessment. If it is >= 0.05, then the comment
                #is perceived as 'positive'. The individual scores show what %
                #of the comment is neutral, +ve, and/or -ve
                score = analyser.polarity_scores(comment.body)
                
                self.comments_dict["comment_id"].append(comment.id)
                self.comments_dict["story_id"].append(submission.id)
                self.comments_dict["comment_body"].append(comment.body)
                self.comments_dict["comment_author"].append(comment.author)
                self.comments_dict["negative_sa_score"].append(score["neg"])
                self.comments_dict["neutral_sa_score"].append(score["neu"])
                self.comments_dict["positive_sa_score"].append(score["pos"])
                self.comments_dict["compound_sa_score"].append(score["compound"])

        self.story_df = pd.DataFrame(self.stories_dict)
        self.story_df = self.story_df.dropna()        
        
        self.comment_df = pd.DataFrame(self.comments_dict)
        self.comment_df = comment_df.dropna()

# Run this section to read stories/comments from the NoSleep Reddit and save them to CSV

In [18]:
nsapp = NoSleepRecommender()
nsapp.getStories()
nsapp.saveAll()

# Run this section to load the stories/comments from the saved CSV files and save them to the DynamoDB tables

In [25]:
nsapp = NoSleepRecommender()
nsapp.loadAll()

comment_df = nsapp.returnComments()
story_df = nsapp.returnStories()

dbeng = DynamoDBEngine()

dbeng.uploadStories(story_df)
dbeng.uploadComments(comment_df)

True

In [23]:
story_df

,story_id,title,author,body
0,fdub8s,February 2020 contest nominations,TheCusterWolf,
1,fecu80,January 2020 Winners!,poppy_moonray,
2,fkszih,How to Survive Camping: cognitive dissonance i...,fainting--goat,I run a private campground. I have a set of r...
3,fkyh67,I started a new job as an overnight security g...,Mr_Mojo_Risin95,"The night dragged on, not seeming to move as I..."
4,fkp50h,The ice cream truck in my neighborhood sells m...,MilesCastle,Only a few of the other kids in my neighborhoo...
5,fkrk6q,"My daughter fell into a well, but I'm not sure...",hyperobscura,I never really spent a lot time with my family...
6,fkr74p,There's Been A String of Suicides in My Town. ...,Worchester_St,[Part 1](https://www.reddit.com/r/nosleep/comm...
7,fkwlz1,Our village used to make the most delicious bu...,likeeyedid,How do you tell if someone is telling a lie? L...
8,fkfyfq,There were stars on the ceiling of my childhoo...,AuthorJoJo,I was always so afraid of the dark as a kid. I...
9,fko21t,I am a glacial researcher and what my team fou...,StairJumper,About twenty-five minutes after Teresa answere...


In [17]:
comment_df

,comment_id,story_id,comment_author,comment_body,negative_sa_score,neutral_sa_score,positive_sa_score,compound_sa_score
0,fkuko2e,fkszih,NoSleepAutoBot,It looks like there may be more to this story....,0.000,0.885,0.115,0.4329
1,fkunkq8,fkszih,A808Ag,While I think it's horrible that they turned o...,0.099,0.642,0.259,0.9449
2,fkuqsqg,fkszih,raeumauf,"So, obviously there is a way of seeing Psychop...",0.096,0.882,0.022,-0.8074
3,fkuwzaj,fkszih,resoredo,Love on first smite\n\nThe ancient being sound...,0.095,0.605,0.300,0.7783
4,fkuxmea,fkszih,RestingBethFace,"So, somewhat unrelated to this specific post, ...",0.038,0.806,0.156,0.8938
...,...,...,...,...,...,...,...,...
201,fkuj9wq,fko21t,XDuVarneyX,Idk what they mean but he describes them like ...,0.089,0.669,0.242,0.4678
202,fkurdo1,fko21t,PingusNoots,"bro Icelanders are fucking crazy, I would know",0.310,0.690,0.000,-0.4005
203,fkv7u3q,fko21t,tech_daddy_dinosaur,For several years .... I wondered what the rea...,0.000,0.751,0.249,0.8074
204,fku83xg,fko21t,ZombieKatanaFaceRR,Ice doesn't form the kind of crystals that wer...,0.000,0.872,0.128,0.3612
